In [300]:
from functools import reduce
from collections import namedtuple
from random import random
from queue import PriorityQueue
from math import ceil

from tqdm.auto import tqdm
import numpy as np

In [301]:
PROBLEM_SIZE = 10
NUM_SETS = 20
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [302]:
#A-star

def h1(state):
    covered_states = covered(state)
    if np.all(covered_states):
        return 0
    pot = map(lambda x: np.logical_or(x, covered_states), state.not_taken)
    best_tilecovering = max(sum(s) for s in pot)
    return len(state.not_taken) - best_tilecovering

def h2(state):
    covered_states = covered(state)
    if np.all(covered_states):
        return 0
    
    


def f_astar(state):
    return len(state.taken) + h1(state)

def f_dijkstra(state):
    return len(state.taken)

def f_greedy(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

In [303]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f_astar(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f_astar(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} sets)")
print("solution: ", [SETS[i] for i in current_state.taken])

0it [00:00, ?it/s]

35it [00:00, 478.47it/s]

Solved in 35 steps (6 sets)
solution:  [array([ True, False, False, False,  True, False, False, False, False,
       False]), array([False, False, False,  True, False, False, False, False,  True,
        True]), array([False, False, False, False, False, False,  True, False, False,
       False]), array([ True, False, False,  True, False, False, False,  True, False,
       False]), array([False, False,  True, False, False,  True, False, False, False,
       False]), array([False,  True, False,  True, False, False, False, False, False,
       False])]


In [304]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f_dijkstra(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f_dijkstra(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} sets)")
print("solution: ", [SETS[i] for i in current_state.taken])

9296it [00:00, 11691.68it/s]

Solved in 9,296 steps (4 sets)
solution:  [array([False, False,  True, False, False,  True, False, False, False,
       False]), array([False,  True, False,  True, False, False, False, False, False,
       False]), array([ True,  True, False, False,  True, False, False, False,  True,
       False]), array([False, False, False, False,  True, False,  True,  True, False,
        True])]
